In [ ]:
# Import All Required Libraries
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed,Conv2D,MaxPooling2D,LSTM,Dense,Dropout,Flatten,GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

In [ ]:
!wget https://www.csc.kth.se/cvap/actions/walking.zip

--2026-01-10 06:32:28--  https://www.csc.kth.se/cvap/actions/walking.zip
Resolving www.csc.kth.se (www.csc.kth.se)... 130.237.28.57, 2001:6b0:1:11c2::82ed:1c39
Connecting to www.csc.kth.se (www.csc.kth.se)|130.237.28.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 242442411 (231M) [application/zip]
Saving to: ‘walking.zip’

walking.zip         100%[===================>] 231.21M   732KB/s    in 5m 38s  

2026-01-10 06:38:08 (700 KB/s) - ‘walking.zip’ saved [242442411/242442411]



In [ ]:
!wget https://www.csc.kth.se/cvap/actions/running.zip

--2026-01-10 06:38:08--  https://www.csc.kth.se/cvap/actions/running.zip
Resolving www.csc.kth.se (www.csc.kth.se)... 130.237.28.57, 2001:6b0:1:11c2::82ed:1c39
Connecting to www.csc.kth.se (www.csc.kth.se)|130.237.28.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149756643 (143M) [application/zip]
Saving to: ‘running.zip’

running.zip         100%[===================>] 142.82M   724KB/s    in 3m 28s  

2026-01-10 06:41:37 (703 KB/s) - ‘running.zip’ saved [149756643/149756643]



In [ ]:
!unzip walking.zip

Archive:  walking.zip
  inflating: person01_walking_d1_uncomp.avi  
  inflating: person01_walking_d2_uncomp.avi  
  inflating: person01_walking_d3_uncomp.avi  
  inflating: person01_walking_d4_uncomp.avi  
  inflating: person02_walking_d1_uncomp.avi  
  inflating: person02_walking_d2_uncomp.avi  
  inflating: person02_walking_d3_uncomp.avi  
  inflating: person02_walking_d4_uncomp.avi  
  inflating: person03_walking_d1_uncomp.avi  
  inflating: person03_walking_d2_uncomp.avi  
  inflating: person03_walking_d3_uncomp.avi  
  inflating: person03_walking_d4_uncomp.avi  
  inflating: person04_walking_d1_uncomp.avi  
  inflating: person04_walking_d2_uncomp.avi  
  inflating: person04_walking_d3_uncomp.avi  
  inflating: person04_walking_d4_uncomp.avi  
  inflating: person05_walking_d1_uncomp.avi  
  inflating: person05_walking_d2_uncomp.avi  
  inflating: person05_walking_d3_uncomp.avi  
  inflating: person05_walking_d4_uncomp.avi  
  inflating: person06_walking_d1_uncomp.avi  
  inflating:

In [ ]:
!unzip running.zip

Archive:  running.zip
  inflating: person01_running_d1_uncomp.avi  
  inflating: person01_running_d2_uncomp.avi  
  inflating: person01_running_d3_uncomp.avi  
  inflating: person01_running_d4_uncomp.avi  
  inflating: person02_running_d1_uncomp.avi  
  inflating: person02_running_d2_uncomp.avi  
  inflating: person02_running_d3_uncomp.avi  
  inflating: person02_running_d4_uncomp.avi  
  inflating: person03_running_d1_uncomp.avi  
  inflating: person03_running_d2_uncomp.avi  
  inflating: person03_running_d3_uncomp.avi  
  inflating: person03_running_d4_uncomp.avi  
  inflating: person04_running_d1_uncomp.avi  
  inflating: person04_running_d2_uncomp.avi  
  inflating: person04_running_d3_uncomp.avi  
  inflating: person04_running_d4_uncomp.avi  
  inflating: person05_running_d1_uncomp.avi  
  inflating: person05_running_d2_uncomp.avi  
  inflating: person05_running_d3_uncomp.avi  
  inflating: person05_running_d4_uncomp.avi  
  inflating: person06_running_d1_uncomp.avi  
  inflating:

In [ ]:
img_size=128
max_frames=20
def load_video(video_pate):
  cap=cv2.VideoCapture(video_pate)
  frames=[]
  while True:
    ret,frame=cap.read()
    if not ret:
      break
    frame=cv2.resize(frame,(img_size,img_size))
    frames.append(frame)
  cap.release()

  frames=np.array(frames)
  if len(frames)>max_frames:
    frames=frames[:max_frames]

  frames=tf.cast(frames,tf.float32)/255.0
  return frames





In [ ]:
X,y=[],[]
for file_name in os.listdir('.'):
  if 'walking' in file_name and file_name.endswith('.avi'):
    video_path=os.path.join('.',file_name)
    video=load_video(video_path)
    X.append(video)
    y.append(0) # Label for walking is 0
  elif 'running' in file_name and file_name.endswith('.avi'):
     video_path=os.path.join('.',file_name)
     video=load_video(video_path)
     X.append(video)
     y.append(1) # Label for running is 1

X=tf.keras.preprocessing.sequence.pad_sequences(X,padding='post',dtype='float32')
y=tf.keras.utils.to_categorical(y,2)

print(X.shape)
print(y.shape)


(200, 20, 128, 128, 3)
(200, 2)


In [ ]:
# Model Creation and adding the Layer

# Create Model and add Layers
model = Sequential([
    TimeDistributed(Conv2D(32, (3,3), activation='relu'), input_shape=(max_frames,128,128,3)),
    TimeDistributed(MaxPooling2D(2,2)),

    TimeDistributed(Conv2D(64, (3,3), activation='relu')),
    TimeDistributed(MaxPooling2D(2,2)),

    TimeDistributed(GlobalAveragePooling2D()),
    Flatten(),

    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_40             │ (None, 20, 126, 126,   │           896 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_41             │ (None, 20, 63, 63, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_42             │ (None, 20, 61, 61, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_43             │ (None, 20, 30, 30, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_44             │ (None, 20, 64)         │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 183,618 (717.26 KB)

 Trainable params: 183,618 (717.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X,y,epochs=10, batch_size=2) # train the model

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.5234 - loss: 0.6905
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6302 - loss: 0.6831
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6810 - loss: 0.6753
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6552 - loss: 0.6665
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.5940 - loss: 0.6826
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6692 - loss: 0.6731
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.6387 - loss: 0.6494
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.4669 - loss: 0.6973
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.4834 - loss: 0.6942
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.5719 - loss: 0.6914


In [43]:
from google.colab import files
uploaded=files.upload()
test_video=list(uploaded.keys())[0]



Saving person01_running_d1_uncomp.avi to person01_running_d1_uncomp (3).avi


In [44]:
test_frames=load_video(test_video)
test_frames=np.expand_dims(test_frames,axis=0)
prediction=model.predict(test_frames)
label=tf.argmax(prediction,axis=1)
print('Final Prediction','Walk' if label==0 else 'Run')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Final Prediction Run
